In [7]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy

###Проведемо А/В тестування впровадження фічі 

необхідно взяти одноріну групу. 
беремо цю групу лише тому, що вважаємо її однорідною з точки зору спожицьких звичок, а не тому що вона купувала ноути

In [10]:
laptop=pd.read_csv('laptop.csv')

In [13]:
laptop.head()

,userid,spending,age,visits
0,31,1499,32,12
1,32,799,23,40
2,33,1200,45,22
3,34,0,59,126
4,35,1350,17,85


розділимо групу на 2 однакові підгрупи випадковим чином

In [116]:
np.random.seed(18811015)
laptop.loc[:,'groupassignment']=1*(np.random.random(len(laptop.index))>0.5)
groupA=laptop.loc[laptop['groupassignment']==0,:].copy()
groupB=laptop.loc[laptop['groupassignment']==1,:].copy()
print(groupA.head())
groupA.shape[0]

   userid  spending  age  visits  groupassignment
0      31      1499   32      12                0
2      33      1200   45      22                0
4      35      1350   17      85                0
5      36      2780   25       6                0
7      38         0   79     450                0


15

In [118]:
print(groupB.head()) 
groupB.shape[0]

    userid  spending  age  visits  groupassignment
1       32       799   23      40                1
3       34         0   59     126                1
6       37      3400   65     428                1
8       39      1800   25     180                1
10      41       999   35     835                1


15

маємо 2 групи по 15 осіб у кожній

проведемо тестування, у якому групі В буде відправлено емейл з впровадженою фічею, а групі А без фічі
візьмемо отримані через деякий час результати тесту

In [19]:
emailresults=pd.read_csv('emailresults2.csv')

In [62]:
emailresults.head()

,userid,revenue
0,31,100
1,32,0
2,33,50
3,34,800
4,35,75


доєднаємо результати тесту до початкових підгруп

In [24]:
groupA_withrevenue=groupA.merge(emailresults,on='userid')
groupB_withrevenue=groupB.merge(emailresults,on='userid')

проведемо наше т-тестування і отримаємо п-значення

In [27]:
scipy.stats.ttest_ind(groupA_withrevenue['revenue'],groupB_withrevenue['revenue'])

TtestResult(statistic=-2.381320497676198, pvalue=0.024288828555138562, df=28.0)

п-значення менше порогового 0,05, тому вважаємо результат статистично значимим (відкидаємо нульову гіпотезу про те, що це 
групи з однієї вибірки).
Резульат - фічу можна впроваджувати і/або продовжувати подальше тестування вже з новою фічею

#### Визначимо ефект і ймовірність успішності нашого тесту

In [78]:
effect = np.mean(groupA_withrevenue['revenue'])-np.mean(groupB_withrevenue['revenue'])
print(np.abs(effect))

260.3333333333333


визначимо розмір еффекту як коефіцієнт Коена д

In [92]:
effect_size = np.abs(effect)/np.std(emailresults['revenue']) 
print(effect_size)

0.8207707199745888


ефект 0,82 по відношенню до стандартоного відхилення вважається значним

In [120]:
from statsmodels.stats.power import TTestIndPower
analysis = TTestIndPower()
alpha = 0.05
power = analysis.solve_power(effect_size=effect_size, nobs1=groupB.shape[0], alpha=alpha)
print(power)

0.5833058537818713


 маємо невисоку ймовірність розрахунку правдивого ефекту 58,3%, не зважаючи на його значну величину. 
 це зумовлено надто малими розмірами вибіркових груп к-стю 15 чоловік.
 нижче покажемо, що збільшивши вибірки хоча б до 30, матимемо значно кращий результат

In [122]:
power = analysis.solve_power(effect_size=effect_size, nobs1=30, alpha=alpha)
print(power)

0.878154655326913


87% можна вважати позитивним результатом.
зробимо зворотній розрахунок, щоб порахувати розмір вибірки, достатній для отримання позитивного тесту з ймовірністю на допустимому рівні 80%

In [128]:
alpha = 0.05
power = 0.8
observations = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha)
print(observations)

24.300763212526185


за ефекту 0,82 нам було б достатньо вибірки у 25 осіб або загальної групи для дослідження 25*2 = 50 осіб